# Makrdown cell點開來有標好TODO要幫忙修改

# Train and Deploy open LLMs with Amazon SageMaker
## TODO: 
### 1. Model要換成phi-3, 要請yuna測試一下幾筆資料可以跑完整個notebook, 不包含deploy
### 2. Deploy這邊要請Shiun來修改
### 3. 這次不會讓他們從hf操作任何東西, 不管是download model還是dataset,可以提一下如果對其他Model或是Dataset有興趣可以怎麼操作 

In this sagemaker example, we are going to learn how to fine-tune open LLMs, like [Mistral](https://huggingface.co/models?other=mistral) using [QLoRA](https://arxiv.org/abs/2305.14314) and how to deploy them afterwards using the <!-- TODO: -->  [Hugging Face LLM Inference DLC](https://huggingface.co/blog/sagemaker-huggingface-llm) @shiun

In our example, we are going to leverage Hugging Face [Transformers](https://huggingface.co/docs/transformers/index), [Accelerate](https://huggingface.co/docs/accelerate/index), and [PEFT](https://github.com/huggingface/peft). We will also make use of new and efficient features and methods including, Flash Attention, Datset Packing and Mixed Precision Training. @richie

## 1. Setup Development Environment

## **設置開發環境**

> 若想在自己的 Hugging Face 下載模型，請登入自己的帳號、產生 token、同意模型使用條款、並修改程式碼。   
> <font color=orange>**由於時間因素，本次工作坊已將模型放上 S3，不會帶大家下載模型**</font>  
> 
> 若想在地端環境使用 SageMaker，需要擁有具備 SageMaker 所需權限的 IAM role，更多資訊請參考 [\[How to use SageMaker execution roles\]](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)

#### **安裝使用 Hugging Face 所需套件**

1. `huggingface_hub`: 用於與 Hugging Face 模型和資料庫互動，允許用戶上傳和下載預訓練的模型、共享和管理模型。
   
2. `transformers`: 用於自然語言處理 (NLP) 任務，支援多種預訓練的模型架構，並提供了加載、微調、和應用模型的工具。
   
3.  `--quiet`: 讓安裝過程中的輸出介面保持整潔，不會印出過多的安裝細節。

In [13]:
!pip install huggingface_hub==0.24.6 transformers==4.44.2 --quiet

#### **設定 SageMaker 環境並獲取相關的 AWS IAM 角色和 S3 bucket 資訊**


In [30]:
import sagemaker
import boto3

# Creates a SageMaker session to manage operations related to SageMaker
sess = sagemaker.Session()

# Setup SageMaker storage bucket:
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

# Get execution role
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Recreates the SageMaker session using the previously obtained sagemaker_session_bucket
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::097724924093:role/bedrock-workshop-studio-v2-SageMakerExecutionRole-2a0B5XslSppz
sagemaker bucket: sagemaker-us-west-2-097724924093
sagemaker session region: us-west-2


## **處理資料集**

我們會用我們預先準備的dataset來處理, 這次dataset是透過ai產的, 我們寫了腳本可以去call bedrock api和claude 3 來幫我們產資料, 腳本可以看 [generate_data.py](./scripts/generate_data.py) 和 [generate_data_2.py](./scripts/generate_data_2.py)

#### **導入預訓練的 tokenizer (分詞器)**

- `AutoTokenizer` 可以自動根據提供的模型 ID 選擇適合的 tokenizer 類別

In [16]:
from transformers import AutoTokenizer

model_id = "microsoft/Phi-3.5-mini-instruct" 
tokenizer = AutoTokenizer.from_pretrained(model_id, token=True)

#### **將本次 workshop 提供的 dataset 載入 SageMaker bucket**

In [22]:
import boto3

s3_client = boto3.client('s3', region_name="us-west-2")

def parse_s3_uri(uri):
    parts = uri.replace("s3://", "").split("/")
    bucket = parts[0]
    key = "/".join(parts[1:])
    return bucket, key


def copy_s3_object(source_uri, target_bucket):
    source_bucket, source_key = parse_s3_uri(source_uri)
    try:
        # Download file from source bucket
        response = s3_client.get_object(Bucket=source_bucket, Key=source_key)
        file_content = response['Body'].read()

        # Upload file to target bucket
        s3_client.put_object(Bucket=target_bucket,
                             Key=source_key, Body=file_content)
        print(f"Successfully copied {source_key} to {target_bucket}")
    except Exception as e:
        print(f"Error copying {source_key}: {str(e)}")


# Base S3 URI for datasets
base_uri = 's3://aws-educate-09-28-sagemaker-workshop-oregon/datasets/phi-3.5-mini-instruct/'
train_uri = base_uri + 'train_dataset.json'
test_uri = base_uri + 'test_dataset.json'

# Define your target S3 bucket
target_bucket = sagemaker_session_bucket

# Copy train and test datasets to the new S3 bucket
copy_s3_object(train_uri, target_bucket)
copy_s3_object(test_uri, target_bucket)

# Construct the S3 URI for the datasets in the target bucket
datasets_s3_uri = f"s3://{target_bucket}/datasets/phi-3.5-mini-instruct/"

Successfully copied datasets/phi-3.5-mini-instruct/train_dataset.json to sagemaker-us-west-2-097724924093
Successfully copied datasets/phi-3.5-mini-instruct/test_dataset.json to sagemaker-us-west-2-097724924093


#### **從 Amazon S3 讀取 JSON 格式的 dataset，並將其格式化為可以使用的 dataset**

In [37]:
import pandas as pd
import json
from datasets import Dataset, DatasetDict

def read_and_format_json_from_s3(s3_uri):

    bucket, key = parse_s3_uri(s3_uri)
    try:
        # Retrieve the object from S3
        obj = s3_client.get_object(Bucket=bucket, Key=key)
        file_content = obj['Body'].read().decode('utf-8')

        # Parse JSON content
        data = []
        try:
            data = json.loads(file_content)
        except json.JSONDecodeError:
            for line in file_content.splitlines():
                try:
                    item = json.loads(line)
                    data.append(item)
                except json.JSONDecodeError:
                    continue

        # Format the data
        formatted_data = []
        for item in data:
            if isinstance(item, dict):
                formatted_item = {}
                if 'messages' in item:
                    formatted_item['input'] = item['messages'][0]['content']
                    formatted_item['output'] = item['messages'][1]['content']
                else:
                    formatted_item = item  # Include the item as is if no messages
                formatted_data.append(formatted_item)

        return formatted_data

    except Exception as e:
        print(f"Error reading from S3: {e}")
        return None
    
# s3 URI
base_uri = 's3://aws-educate-09-28-sagemaker-workshop-oregon/datasets/phi-3/'
train_uri = base_uri + 'train_dataset.json'
test_uri = base_uri + 'test_dataset.json'

# Load and format training and test data from S3
train_data = read_and_format_json_from_s3(train_uri)
test_data = read_and_format_json_from_s3(test_uri)

# Create DatasetDict from the formatted data
dataset = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(train_data)),
    'test': Dataset.from_pandas(pd.DataFrame(test_data))
})

# Print dataset information
print(f"Dataset Info:\n{dataset}")
print(f"\nDataset Features:\n{dataset['train'].features}")


Dataset Info:
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 6776
    })
    test: Dataset({
        features: ['text'],
        num_rows: 753
    })
})

Dataset Features:
{'text': Value(dtype='string', id=None)}


#### **將訓練和測試資料保存到指定的文件路徑中**

In [38]:
import json
import os

# Define the directory to save the datasets
save_path = '/opt/ml/input/data/training'
os.makedirs(save_path, exist_ok=True)


def save_dataset(file_path, data):
    with open(file_path, 'w') as f:
        for item in data:
            json.dump(item, f)
            f.write('\n')


# Save training data
train_file_path = os.path.join(save_path, 'train_dataset.json')
save_dataset(train_file_path, train_data)

# Save test data
test_file_path = os.path.join(save_path, 'test_dataset.json')
save_dataset(test_file_path, test_data)

print(f"Dataset saved to: {save_path}")

Dataset saved to: /opt/ml/input/data/training


#### **嘗試讀取 S3 中的 dataset (output.json) 並顯示結果**

In [41]:
import boto3
import json
from botocore.exceptions import ClientError

def read_and_display_s3_data(s3_uri, limit=3):
    # Parse the S3 URI into bucket and key
    bucket, key = parse_s3_uri(s3_uri)

    try:
        # Retrieve the object from S3
        response = s3_client.get_object(Bucket=bucket, Key=key)
        
        # Read and decode the data
        data = response['Body'].read().decode('utf-8')

        # Parse the JSON data
        json_data = json.loads(data)
        
        # Display a limited number of conversations
        for idx, item in enumerate(json_data[:limit], 1):
            print(f"\nConversation {idx}:")
            for message in item.get('messages', []):
                role = message.get('role', 'unknown').capitalize()
                content = message.get('content', 'No content')
                print(f"  {role}: {content}")

    except ClientError as e:
        print(f"Error reading S3 data: {e}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON data: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example usage
output_uri = base_uri + 'output.json'
read_and_display_s3_data(output_uri)


Conversation 1:
  User: 看個股票算算自己又賺了多少
  Assistant: 喵～看來你是個精打細算的小喵星人呢！EC2就像你的個人計算機，隨時待命喵。不過要小心別太著迷於數字遊戲喔，別忘了真正的財富是那些無價的喵喵時光～(=ΦωΦ=)

Conversation 2:
  User: 光速完食立馬出門搭車
  Assistant: 哇喔！你簡直就是Lambda函數啊！迅速執行任務，不拖泥帶水。不過別忘了，就算是最高效的程式也需要適當的休息和錯誤處理喵～(ΦзΦ) 偶爾慢下來品嚐早餐的美好，說不定會有意想不到的收穫喔！

Conversation 3:
  User: 突然想發個限動
  Assistant: 喵嗚～看來你是個愛炫耀的小可愛呢！就像DynamoDB一樣，隨時準備好存儲和分享你的精彩時刻喵。不過要記得，真正的朋友不會因為你沒發限動就忘記你的存在喔～(=^ω^=) 有時候，最珍貴的回憶是藏在心裡的那些呢！


## **進行 Fine-Tune**

#### **設置超參數**

In [42]:
from huggingface_hub import HfFolder


# hyperparameters, which are passed into the training job
hyperparameters ={
  'model_id': model_id,                             # pre-trained model
  'dataset_path': '/opt/ml/input/data/training',    # path where sagemaker will save training dataset
  'num_train_epochs': 3,                            # number of training epochs
  'per_device_train_batch_size': 1,                 # batch size for training
  'gradient_accumulation_steps': 2,                 # Number of updates steps to accumulate 
  'gradient_checkpointing': True,                   # save memory but slower backward pass
  'fp16': True ,
  'learning_rate': 2e-4,                            # learning rate
  'max_grad_norm': 0.3,                             # Maximum norm (for gradient clipping)
  'warmup_ratio': 0.03,                             # warmup ratio
  "lr_scheduler_type":"constant",                   # learning rate scheduler
  'save_strategy': "epoch",                         # save strategy for checkpoints
  "logging_steps": 10,                              # log every x steps
  'merge_adapters': True,                           # wether to merge LoRA into the model (needs more memory)
  'use_flash_attn': True,                           # Whether to use Flash Attention
  'output_dir': '/tmp/run',                         # output directory, where to save assets during training
}

if HfFolder.get_token() is not None:
    hyperparameters['hf_token'] = HfFolder.get_token() # huggingface token to access gated models, e.g. llama 2

#### **設置訓練任務**

In [43]:
from sagemaker.huggingface import HuggingFace

# define Training Job Name 
job_name = f'huggingface-qlora-{hyperparameters["model_id"].replace("/","-").replace(".","-")}'

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_qlora.py',    # train script
    source_dir           = '../scripts',      # directory which includes all the files needed for training
    instance_type        = 'ml.p3.2xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    max_run              = 2*24*60*60,        # maximum runtime in seconds (days * hours * minutes * seconds)
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.36',            # the transformers version used in the training job
    pytorch_version      = '2.1',             # the pytorch_version version used in the training job
    py_version           = 'py310',           # the python version used in the training job
    hyperparameters      =  hyperparameters,  # the hyperparameters passed to the training job
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache }, # set env variable to cache models in /tmp
    disable_output_compression = True         # not compress output to save training time and cost
)

#### **設定訓練數據的位置，並啟動 Hugging Face 模型的訓練任務**

In [ ]:
# Define a data input dictionary with the uploaded S3 URIs
data = {
    'training': f"s3://{sess.default_bucket()}/datasets/phi-3.5-mini-instruct/"
}

# Start the training job using the provided datasets as input
huggingface_estimator.fit(data, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-qlora-microsoft-Phi-3-5-min-2024-09-12-22-02-00-505


2024-09-12 22:02:01 Starting - Starting the training job
2024-09-12 22:02:01 Pending - Training job waiting for capacity...
2024-09-12 22:02:30 Pending - Preparing the instances for training.

#### **將 S3 的 URI 轉換為一個可以直接在 AWS S3 管理控制台中訪問的 URL**

In [ ]:
huggingface_estimator.model_data["S3DataSource"]["S3Uri"].replace("s3://", "https://s3.console.aws.amazon.com/s3/buckets/")

## 🚀<font color=orange>**Now, lets deploy our model to an endpoint. 🚀**</font>

-----
## <font color=red>**以下為存檔**</font>
.   
.   
.   
.   
.   
.   
.   
.   
.   
.   
.   
-----

## 3. Fine-Tune Mistral 7B with QLoRA on Amazon SageMaker
## TODO
### 一樣model要改, 這次訓練腳本確實是用run_qlora

We are going to use the recently introduced method in the paper "[QLoRA: Quantization-aware Low-Rank Adapter Tuning for Language Generation](https://arxiv.org/abs/2305.14314)" by Tim Dettmers et al. QLoRA is a new technique to reduce the memory footprint of large language models during finetuning, without sacrificing performance. The TL;DR; of how QLoRA works is: 

* Quantize the pretrained model to 4 bits and freezing it.
* Attach small, trainable adapter layers. (LoRA)
* Finetune only the adapter layers, while using the frozen quantized model for context.

We prepared a [run_qlora.py](./scripts/run_qlora.py), which implements QLora using PEFT to train our model. The script also merges the LoRA weights into the model weights after training. That way you can use the model as a normal model without any additional code. The model will be temporally offloaded to disk, if it is too large to fit into memory.

In Addition to QLoRA we will leverage the new [Flash Attention 2 integrationg with Transformers](https://huggingface.co/docs/transformers/perf_infer_gpu_one#flash-attention-2) to speed up the training. Flash Attention 2 is a new efficient attention mechanism that is up to 3x faster than the standard attention mechanism. 

In [ ]:
from huggingface_hub import HfFolder


# hyperparameters, which are passed into the training job
hyperparameters ={
  'model_id': model_id,                             # pre-trained model
  'dataset_path': '/opt/ml/input/data/training',    # path where sagemaker will save training dataset
  'num_train_epochs': 3,                            # number of training epochs
  'per_device_train_batch_size': 1,                 # batch size for training
  'gradient_accumulation_steps': 2,                 # Number of updates steps to accumulate 
  'gradient_checkpointing': True,                   # save memory but slower backward pass
  'fp16': True ,
  'learning_rate': 2e-4,                            # learning rate
  'max_grad_norm': 0.3,                             # Maximum norm (for gradient clipping)
  'warmup_ratio': 0.03,                             # warmup ratio
  "lr_scheduler_type":"constant",                   # learning rate scheduler
  'save_strategy': "epoch",                         # save strategy for checkpoints
  "logging_steps": 10,                              # log every x steps
  'merge_adapters': True,                           # wether to merge LoRA into the model (needs more memory)
  'use_flash_attn': True,                           # Whether to use Flash Attention
  'output_dir': '/tmp/run',                         # output directory, where to save assets during training
}

if HfFolder.get_token() is not None:
    hyperparameters['hf_token'] = HfFolder.get_token() # huggingface token to access gated models, e.g. llama 2

In order to create a sagemaker training job we need an `HuggingFace` Estimator. The Estimator handles end-to-end Amazon SageMaker training and deployment tasks. The Estimator manages the infrastructure use. Amazon SagMaker takes care of starting and managing all the required ec2 instances for us, provides the correct huggingface container, uploads the provided scripts and downloads the data from our S3 bucket into the container at `/opt/ml/input/data`. Then, it starts the training job by running.

> Note: Make sure that you include the `requirements.txt` in the `source_dir` if you are using a custom training script. We recommend to just clone the whole repository.

In [ ]:
from sagemaker.huggingface import HuggingFace

# define Training Job Name 
job_name = f'huggingface-qlora-{hyperparameters["model_id"].replace("/","-").replace(".","-")}'

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_qlora.py',    # train script
    source_dir           = '../scripts',      # directory which includes all the files needed for training
    #TODO: 這邊可以確定一下還有哪些instace可以用 https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-available-instance-types.html
    instance_type        = 'ml.p3.2xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    max_run              = 2*24*60*60,        # maximum runtime in seconds (days * hours * minutes * seconds)
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.36',            # the transformers version used in the training job
    pytorch_version      = '2.1',             # the pytorch_version version used in the training job
    py_version           = 'py310',           # the python version used in the training job
    hyperparameters      =  hyperparameters,  # the hyperparameters passed to the training job
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache" }, # set env variable to cache models in /tmp
    disable_output_compression = True         # not compress output to save training time and cost
)

> You can also use `g5.2xlarge` instead of the `g5.4xlarge` instance type, but then it is not possible to use `merge_weights` parameter, since to merge the LoRA weights into the model weights, the model needs to fit into memory. But you could save the adapter weights and merge them using [merge_adapter_weights.py](./scripts/merge_adapter_weights.py) after training.

We can now start our training job, with the `.fit()` method passing our S3 path to the training script.

In [ ]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)

In our example for Mistral 7B, the SageMaker training job took `13968 seconds`, which is about `3.9 hours`. The ml.g5.4xlarge instance we used costs `$2.03 per hour` for on-demand usage. As a result, the total cost for training our fine-tuned Mistral model was only ~`$8`. 

Now lets make sure SageMaker has successfully uploaded the model to S3. We can use the `model_data` property of the estimator to get the S3 path to the model. Since we used `merge_weights=True` and `disable_output_compression=True` the model is stored as raw files in the S3 bucket. 

In [ ]:
huggingface_estimator.model_data["S3DataSource"]["S3Uri"].replace("s3://", "https://s3.console.aws.amazon.com/s3/buckets/")

You should see a similar folder structure and files in your S3 bucket:

![S3 Bucket](../assets/s3.png)

Now, lets deploy our model to an endpoint. 🚀

In [ ]:
# 到這邊為止

## 部署模型


In [ ]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  # version="1.1.0",
  session=sess,
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

We can now create a `HuggingFaceModel` using the container uri and the S3 path to our model. We also need to set our TGI configuration including the number of GPUs, max input tokens. You can find a full list of configuration options [here](https://huggingface.co/docs/text-generation-inference/basic_tutorials/launcher).

In [ ]:
import json
from sagemaker.huggingface import HuggingFaceModel

# s3 path where the model will be uploaded
# if you try to deploy the model to a different time add the s3 path here
model_s3_path = huggingface_estimator.model_data["S3DataSource"]["S3Uri"]

# sagemaker config
instance_type = "ml.g5.2xlarge"
number_of_gpu = 1
health_check_timeout = 300

# Define Model and Endpoint configuration parameter
config = {
  'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
  'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
  'MAX_INPUT_LENGTH': json.dumps(1024), # Max length of input text
  'MAX_TOTAL_TOKENS': json.dumps(2048), # Max length of the generation (including input text)
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  model_data={'S3DataSource':{'S3Uri': model_s3_path,'S3DataType': 'S3Prefix','CompressionType': 'None'}},
  env=config
)

After we have created the HuggingFaceModel we can deploy it to Amazon SageMaker using the deploy method.

In [ ]:

# Deploy model to an endpoint
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout, # 10 minutes to be able to load the model
)

SageMaker will now create our endpoint and deploy the model to it. This can takes a 10-15 minutes.


In [ ]:
llm.delete_model()
llm.delete_endpoint()